In [ ]:
%matplotlib inline
import numpy
import numpy as np
from matplotlib import pyplot
from amuse.units import (units, constants)
from amuse.ext.orbital_elements import new_binary_from_orbital_elements
from amuse.ext.orbital_elements import orbital_elements_from_binary
from amuse.lab import Particles
from amuse.lab import new_powerlaw_mass_distribution
from amuse.lab import nbody_system
from amuse.community.ph4.interface import ph4
from amuse.ext.LagrangianRadii import LagrangianRadii
def orbital_period(Mtot, a):
    return (((4 * numpy.pi**2) * a**3)/(constants.G * Mtot)).sqrt()

SMBH = Particles(1)
SMBH.name = "SMBH"
SMBH.mass = 4.154e+6 | units.MSun
SMBH.position = (0, 0, 0) | units.pc
SMBH.velocity = (0, 0, 0) | units.kms

In [ ]:
def relative_orbital_velocity(distance, mass=SMBH.mass):
    return (constants.G*mass/distance).sqrt()

def dist(body):
    return (body.x**2+body.y**2+body.z**2).sqrt()

def plot(body):
    scatter(body.x.value_in(units.pc), body.z.value_in(units.pc), s=20, alpha=0.5)
    #scatter(SMBH.x.value_in(units.pc), SMBH.z.value_in(units.pc), s=80, c='r')
    pyplot.xlim(-10, 10)
    pyplot.ylim(-10, 10)
    
def random_p(r_in=0, r_out=3):
    r=r_in + np.random.rand()*(r_out-r_in)
    a=np.random.rand()*2*np.pi
    x=r*np.cos(a) | units.pc
    y=r*np.sin(a) | units.pc
    z = 0 | units.pc
    return x,y,z
    
def random_v(distance):
    v=(np.random.rand()*(2*constants.G*SMBH.mass/distance)).sqrt()
    a=np.random.rand()*2*np.pi
    vx=v*np.cos(a)
    vy=v*np.sin(a)
    vz=0 | units.kms
    return vx,vy,vz

In [ ]:
for i in range(0, 1):
    bodies = Particles(2)
    bodies.name = ["BH1", "BH2"]
    bodies.mass = new_powerlaw_mass_distribution(2, 1.0|units.MSun, 100.0|units.MSun, 2.35)
    bodies[0].position = (random_p())
    bodies[1].position = (random_p())
    distance1, distance2 = dist(bodies[0]), dist(bodies[1])
    bodies[0].velocity = (random_v(distance1))
    bodies[1].velocity = (random_v(distance2))
    bodies.add_particles(SMBH)
    
    bodies1 = bodies.copy()
    converter = nbody_system.nbody_to_si(bodies1.mass.sum(), bodies1.position.length())
    bodies1.scale_to_standard(converter)
    gravity = ph4(converter, number_of_workers=32)
    gravity.particles.add_particles(bodies1)
    channel = gravity.particles.new_channel_to(bodies1)


    times = numpy.arange(0, 100, 1) | units.Myr
    for time in times:
        gravity.evolve_model(time)
        channel.copy() # Copy from gravity.particles to bodies
        print("Time=", time.in_(units.Myr))

        b = (bodies-SMBH).get_binaries()
        if(len(b)>0):
            print("Number of binaries found:", len(b))
            print("first binary:", b[0].mass)

In [ ]:
print(bodies1)